In [0]:

from pyspark.sql.functions import row, col, to_date, hour, minute

# Reload from silver
df_cleaned = spark.table("default.parking_cleaned")

# lag dim_parkering

# Hent eksisterende tabell hvis den finnes, ellers None
existing_dim_parkering = spark.table("default.dim_parkering") if "default.dim_parkering" in [t.name for t in spark.catalog.listTables("default")] else None

# Finn nye unike parkeringsplasser og gi kolonnene mer beskrivende navn
new_dim_parkering = df_cleaned.select("Sted", "Latitude", "Longitude").distinct()
new_dim_parkering = new_dim_parkering.withColumnRenamed("Sted", "Parkering_navn")


# Bruk MERGE for å oppdatere dimensjonstabellen uten duplikater

# gir lik funksjonalitet som å bruke DataFrame-basert delta merge med DeltaTable.forName(...).alias(...).merge(...)
# kun forskjell i syntaks

spark.sql("""
MERGE INTO default.dim_parkering AS target
USING (SELECT DISTINCT Sted AS Parkering_navn, Latitude, Longitude FROM default.staging_parking) AS source
ON target.Parkering_navn = source.Parkering_navn
WHEN NOT MATCHED THEN INSERT *
""")


# dim_tid

# Bygg batch kun basert på nyeste data
dim_tid_batch = df_cleaned.select(
    to_date("timestamp").alias("dato"),
    hour("timestamp").alias("time"),
    minute("timestamp").alias("minutt")
).distinct()


# gjør tabellen tilgjengelig for Spark SQL
dim_tid_batch.createOrReplaceTempView("new_dim_tid")


# MERGE for å legge til nye tidspunkter uten duplikater

# # gir lik funksjonalitet som å bruke DataFrame-basert delta merge med DeltaTable.forName(...).alias(...).merge(...)
# kun forskjell i syntaks
# alle delta tables blir lest inn i Spark som DataFrames når man gjør spørringer på dem


spark.sql("""
MERGE INTO default.dim_tid AS target
USING new_dim_tid AS source
ON target.dato = source.dato AND target.time = source.time AND target.minutt = source.minutt
WHEN NOT MATCHED THEN INSERT *
""")


